In [1]:
import sys
import numpy as np
from qiskit import *

sys.path.append('..')
from pyquopt import *

### Define Circuit of Interest

In [2]:
cnot = QuantumCircuit(2)
cnot.cx(1, 0)
cnot.draw()

┌───┐
q_0: ┤ X ├
     └─┬─┘
q_1: ──■──

In [3]:
cnot_transpiled = transpile(cnot, basis_gates=['sx', 'rz', 'cx'], optimization_level=3)

In [4]:
cnot_transpiled.draw()

┌───┐
q_0: ┤ X ├
     └─┬─┘
q_1: ──■──

### Set Optimization Hyperparameters

In [71]:
num_qubits = 2
num_mq_instructions = 1  # want to explore circuits with one multi-qubit gate
num_params = 3 * num_qubits * (num_mq_instructions + 1)  # formula explained in MICRO paper

mq_dict = {
    0: TwoGates.CR01,
}

In [102]:
alpha = 0.30  # penalty for non-standard angles
gamma = 1e-5   # penalty for large angles
non_fixed_params = np.array([1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1]) # leave all parameters unfixed
fixed_params_vals = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])  # unfixed parameters should have a "fixed" value of 0

In [103]:
mq_instructions = [0]

### Solve Optimization Problem

In [104]:
optimizer = Optimizer(num_qubits=num_qubits, mq_instructions=mq_instructions, mq_dict=mq_dict, target=TwoGates.CX10,
                     alpha=alpha, gamma=gamma, non_fixed_params=non_fixed_params, fixed_params_vals=fixed_params_vals)

In [105]:
opt_params, opt_val = optimizer.find_parameters_least_squares(20)

In [106]:
opt_params

array([ 1.57079633e+00,  3.14159266e+00,  1.57079634e+00,  1.57079633e+00,
        3.16192764e+00,  3.14159266e+00, -1.57079633e+00,  0.00000000e+00,
        9.36940866e-09,  1.57079633e+00,  6.28318527e+00, -1.59113130e+00])

In [107]:
opt_params_rounded = round_params(opt_params)
print(opt_params_rounded)

[ 90. 180.  90.  90. 180. 180. 270.   0.   0.  90.   0. 270.]


### Verify Implementation Accuracy

In [108]:
ub = UnitaryBuilder(num_qubits, mq_instructions, mq_dict)

In [109]:
implementation_matrix = ub.build_unitary(opt_params)

In [110]:
print(get_unitary_infidelity(TwoGates.CX10, implementation_matrix, 4))

4.440892098500626e-16


In [111]:
implementation_matrix

array([[ 1.00000000e+00-1.42550592e-08j, -2.01225399e-08+1.72467385e-10j,
        -7.75156901e-10+4.29672398e-09j, -3.44769371e-10+1.57743971e-09j],
       [ 3.44769411e-10+1.57743989e-09j,  1.88714316e-11+4.29672391e-09j,
         2.01225399e-08+1.72467164e-10j,  1.00000000e+00+1.68182426e-09j],
       [ 7.75157014e-10+4.29672386e-09j, -3.44769442e-10+1.57743977e-09j,
         1.00000000e+00-1.71861262e-09j, -2.01225398e-08+1.72467155e-10j],
       [ 2.01225399e-08+1.72466946e-10j,  1.00000000e+00-4.52054644e-09j,
         3.44769395e-10+1.57743990e-09j, -1.88713511e-11+4.29672380e-09j]])